In [1]:
import re

In [2]:
# Define paths
r_script_path = "/data/gardeux/asap_run/install_packages.R" # Old script, packages are sorted in the right dependencies order
output_r_script_path = "/data/gardeux/asap_run/install_packages_updated.R"  # Updated script that will be generated by this script
installed_versions_path = "/data/gardeux/asap_run/.R_packages_version.v8.txt" # Versions to update to

In [3]:
# Load installed package versions
installed_versions = {}
with open(installed_versions_path, "r") as f:
    for line in f:
        match = re.search(r"/([^/]+)/DESCRIPTION:Version: ([\d.-]+)", line)
        if match:
            package, version = match.groups()
            installed_versions[package] = version
installed_versions

{'datasets': '4.4.3',
 'compiler': '4.4.3',
 'mgcv': '1.9-1',
 'cluster': '2.1.8.1',
 'lattice': '0.22-6',
 'methods': '4.4.3',
 'Matrix': '1.7-3',
 'nnet': '7.3-20',
 'utils': '4.4.3',
 'codetools': '0.2-20',
 'KernSmooth': '2.23-26',
 'grid': '4.4.3',
 'survival': '3.8-3',
 'boot': '1.3-31',
 'grDevices': '4.4.3',
 'splines': '4.4.3',
 'spatial': '7.3-18',
 'foreign': '0.8-88',
 'stats4': '4.4.3',
 'MASS': '7.3-65',
 'class': '7.3-23',
 'tcltk': '4.4.3',
 'base': '4.4.3',
 'graphics': '4.4.3',
 'parallel': '4.4.3',
 'translations': '4.4.3',
 'tools': '4.4.3',
 'stats': '4.4.3',
 'nlme': '3.1-167',
 'rpart': '4.1.24',
 'loomR': '0.2.1.9000',
 'rhdf5': '2.51.2',
 'backports': '1.5.0',
 'numDeriv': '2016.8-1.1',
 'tensorA': '0.36.2.1',
 'QuickJSR': '1.6.0',
 'inline': '0.3.21',
 'RcppParallel': '5.1.10',
 'Formula': '1.2-5',
 'bdsmatrix': '1.3-7',
 'checkmate': '2.3.2',
 'distributional': '0.5.0',
 'StanHeaders': '2.32.10',
 'rstantools': '2.4.0',
 'bbmle': '1.0.25.1',
 'posterior': '1.

In [4]:
# Read the original R script
with open(r_script_path, "r") as f:
    r_script_lines = f.readlines()

# Update the script
updated_script = []
cran_pattern = re.compile(r'install\.cran\("(.*?)", version="(.*?)"\)')
bioc_pattern = re.compile(r'install\.bioconductor\("(.*?)", version="(.*?)"')

for line in r_script_lines:
    cran_match = cran_pattern.search(line)
    bioc_match = bioc_pattern.search(line)
    
    if cran_match:
        package, old_version = cran_match.groups()
        new_version = installed_versions.get(package)
        if new_version:
            line = f'install.cran("{package}", version="{new_version}")\n'
        else:
            line = f'# {line.strip()}  # Package version not found\n'
    elif bioc_match:
        package, old_version = bioc_match.groups()
        new_version = installed_versions.get(package)
        if new_version:
            line = re.sub(f'version="{old_version}"', f'version="{new_version}"', line)
        else:
            line = f'# {line.strip()}  # Package version not found\n'
    
    updated_script.append(line)

# Write the updated script
with open(output_r_script_path, "w") as f:
    f.writelines(updated_script)

print(f"Updated R script saved to {output_r_script_path}")

Updated R script saved to /data/gardeux/asap_run/install_packages_updated.R
